In [1]:
import EAC
import os 
import pandas as pd
import sqlite3

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
training = EAC.training()

conn = sqlite3.connect('training.db')

In [ ]:
create_meso_q = '''
CREATE TABLE upcomingTraining (
week int,
session char(20),
movement char(20),
method char(20),
sets int,
reps int,
weight float(4),
progNotes char(140),
targetReps int
)
'''

In [ ]:
#each movement/lift is the pk
create_movement_table_q = '''
CREATE TABLE movements (
movement CHAR(40) PRIMARY KEY not null,
split CHAR(10),
acutalMax int,
repMaxCalc float(4),
veloEquation CHAR(30),
kilos binary
);

'''

In [33]:
#methodID is a amalgamation of method and week 
create_program_builder_q = '''
CREATE TABLE program_builder (
methodID char(30) PRIMARY KEY not null,
method char(20),
week int,
sets int,
reps int,
weight float(4),
notes char(140),
targetReps int
);
'''

In [ ]:
#sessionID  = date_exercise
create_history_q = '''
CREATE TABLE history(
sessionID PRIMARY KEY not null,
sessionDate date,
session char(30),
movement CHAR(40),
method CHAR(20),
setsCompleted int,
repsCompleted int,
weightUsed int,
amrapTotal int,
newMax binary,
notes CHAR(200),
RepCalcM int);
'''

In [ ]:
#veloID = date_exercise_set_rep
create_velo_q='''
CREATE TABLE velo (
movement char(40),
sessionDate date,
weight_kg float(4),
weight_lb float(4),
set_ct int,
rep int,
ecc_con char(4),
MPV_ms float(4),
peak_velo float(4),
rom_cm float(2),
rom_in float(2),
watts int,
est_max_kg float(4),
est_max_lb float(4)


)
'''

In [ ]:
create_rpe_q = '''
CREATE TABLE rpe (
RPE float(1) PRIMARY KEY not null,
rep1 float(4),
rep2 float(4),
rep3 float(4),
rep4 float(4),
rep5 float(4),
rep6 float(4),
rep7 float(4),
rep8 float(4),
rep9 float(4),
rep10 float(4),
rep11 float(4),
rep12 float(4),
rep13 float(4),
rep14 float(4),
rep15 float(4),
rep16 float(4)
);'''

In [35]:
conn.execute('DROP TABLE program_builder')
conn.execute(create_program_builder_q)
# conn.execute(create_meso_q)
# conn.execute(create_rpe_q)
# conn.execute(create_history_q)
# conn.execute(create_movement_table_q)
# conn.execute(create_velo_q)
conn.commit()


### Data Loading

In [25]:
path = os.getcwd()+'/InitialData'

In [26]:
pbFile = [f for f in os.scandir(path) if f.name.find('Builder')>-1][0]
mesoFile = [f for f in os.scandir(path)  if f.name.find(' Meso')!=-1][0]
startDate = mesoFile.name.split(' - ')[0][-8:]
weeks = mesoFile.name.split(' - ')[1][9:11]
setup = mesoFile.name.split(' - ')[1][12:-4]


In [108]:
def create_upcoming_training(file,connection):
    df = pd.DataFrame(columns = ['week','session','movement','method','sets','reps','weight','notes','targetReps'])
    prgbld = pd.read_sql('''
    select * 
    from program_builder
    ''',conn)
    mesoDF = pd.read_csv(file).fillna(0)
    v = df.columns[-5:].values
    r=0
    for sesh in mesoDF.session.unique():
        d = mesoDF[mesoDF.session == sesh]
        for movement in d.exercise.unique():
            endweek = d[d.exercise==movement].endWeek.values[0]
            mod = d[d.exercise==movement].modality.values[0]
            cyclelength = prgbld[prgbld.method==mod].week.max()
            setAdjust = d[d.exercise==movement].SetAdjust.values[0]
            print(movement,mod,setAdjust)
            for i in range(endweek):
                lkup = i+1 if cyclelength==endweek else (i+1) % cyclelength
                lkup = cyclelength if lkup == 0 else lkup
                l = [i+1,sesh,movement,mod]+prgbld[(prgbld.method==mod) & (prgbld.week==lkup)].filter(v).values.tolist()[0]
                df.loc[len(df)] = l
                df.at[r,'sets'] = df.at[r,'sets'] + setAdjust
                r+=1
    add_data(df.values,'upcomingTraining',conn)    
    print('upcomingTraining table has been updated with {}'.format(file))

In [64]:
def add_data(data,table,connection):
    '''Pass through a list or array of values to be written into database'''
    qs = '?'+(len(data[0])-1)*',?'
    with connection as conn:
        conn.executemany('''INSERT into {t}
                        VALUES ({row})'''.format(t=table,row=qs),data)

In [61]:
pbDF = pd.read_csv(pbFile.path)

In [71]:
add_data(pbDF.values,'program_builder',conn)

INSERT into program_builder
                        VALUES (['?,?,?,?,?,?,?,?'])


In [109]:
create_upcoming_training(mesoFile.path,conn)

bench 5x531 0.0
kab bench Tech 0.0
db incline Supp 0.0
Squat wendDouble 0.0
Front Tech 0.0
Wide Box Hyp 0.0
Deadlift  wendDouble 0.0
Trap Str 0.0
Meadows Supp 0.0
Seal Hyp 1.0
Bandbell standing press Supp 1.0
Floor Supp 1.0
Box Str 0.0
goodMorning Hyp 0.0
Pendlay row Hyp 1.0
t-bar row Hyp 1.0
INSERT into upcomingTraining
                        VALUES (['?,?,?,?,?,?,?,?,?'])
upcomingTraining table has been updated with /Users/ericcoxon/code/lifts/site/InitialData/MesoLoad_06022023 - MesoLoad_18_PPL.csv
